# TODO


1.   Натренировать Relation Extraction
2.   Добавить early_stopping
3.   Save-n models
5.   Совместить с NEREL

# DONE

In [1]:
ls

Miniconda3-latest-Linux-x86_64.sh*  multilabel_ner/
bert_rured_training/                onstart.sh
bert_rured_training.zip             pip.log
hasbooted                           rured2/
id_to_label.json                    rured_nested_training.ipynb
miniconda3/


In [2]:
!apt install zip -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
zip is already the newest version (3.0-11build1).
0 upgraded, 0 newly installed, 0 to remove and 10 not upgraded.


In [3]:
!git clone https://github.com/denis-gordeev/rured2

fatal: destination path 'rured2' already exists and is not an empty directory.


In [4]:
!nvidia-smi

Tue Nov 14 21:00:57 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080        On  | 00000000:42:00.0 Off |                  N/A |
|  0%   32C    P8              23W / 370W |      2MiB / 10240MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
!pip install torch >> ./pip.log
!pip install razdel==0.5.0 >> ./pip.log
!pip install seqeval==1.2.2 >> ./pip.log
!pip install datasets==2.2.2 >> ./pip.log
!pip install fire==0.4.0 >> ./pip.log
!pip install boto3==1.24.6 >> ./pip.log
!pip install transformers[torch] >> ./pip.log
!pip install sentencepiece >> ./pip.log
!pip install protobuf >> ./pip.log
!pip install accelerate -U >> ./pip.log

In [6]:
!ls rured2/data/annotation_files | head
!echo "Total $(ls rured2/data/annotation_files | wc -l) files"

101144.ann
101144.txt
102083.ann
102083.txt
10223.ann
10223.txt
104927.ann
104927.txt
11035.ann
11035.txt
ls: write error: Broken pipe
Total 2304 files


In [7]:
from glob import glob
import shutil

In [8]:
# I have mixed automatically and manually labelled files here
# Currently only manually annotated files contain relations
ann_files = glob("rured2/data/rbc/*/*.ann")
for ann_file in ann_files:
    with open(ann_file) as f:
        annotation = f.readlines()
    if not any(l.startswith("R") for l in annotation):
        continue
    txt_file = ann_file.replace(".ann", ".txt")
    shutil.copy(ann_file, "rured2/data/annotation_files/")
    shutil.copy(txt_file, "rured2/data/annotation_files/")

In [9]:
!ls rured2/data/annotation_files | head
!echo "Total $(ls rured2/data/annotation_files | wc -l) files"

101144.ann
101144.txt
102083.ann
102083.txt
10223.ann
10223.txt
104927.ann
104927.txt
11035.ann
11035.txt
ls: write error: Broken pipe
Total 2304 files


In [10]:
# !git clone --recurse-submodules https://github.com/denis-gordeev/rured/

In [11]:
# ls ./rured/SpanBERT/

In [12]:
# !python ./rured/convert_brat_to_tacred.py "rured2/data/annotation_files/*.ann"

In [13]:
# ls -lah all_data.json

In [14]:
from glob import glob
from typing import Dict, List, Optional

import pandas as pd
from sklearn.model_selection import train_test_split
import razdel
import torch
from torch import nn
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, top_k_accuracy_score, recall_score, precision_score

from datasets import Dataset
from transformers import (AutoTokenizer, AutoModelForTokenClassification, 
                          TrainingArguments, Trainer)
from transformers import EarlyStoppingCallback

In [15]:
# model_ckpt = "cointegrated/rubert-tiny2"
# model_ckpt = "yandex/RuLeanALBERT"
# model_ckpt = "xlm-roberta-base"
# model_ckpt = "bert-base-multilingual-cased"
# model_ckpt = "google/flan-t5-base"
model_ckpt = "microsoft/mdeberta-v3-base"

In [16]:
def check_pair_in_range(start: int, end: int, check_range: range) -> bool:
    if range(max(start, check_range.start), min(end, check_range.stop)):
        return True
    else:
        return False

def get_annotation_from_file(
    annotation_filename: str, with_relations_only: bool = False
) -> List[dict]:
    with open(annotation_filename) as f:
        annotation_str: str = f.read()
    annotation_values: List[dict] = []
    if with_relations_only and "Arg1:" not in annotation_str:
        return []
    annotation: List = annotation_str.split("\n")
    annotation = [a.strip() for a in annotation]
    annotation = [a for a in annotation if a]
    annotation = [a.split("\t") for a in annotation]
    annotation_entries: dict = dict()
    ners = [a for a in annotation if a[0][0] == "T"]
    relations = [a for a in annotation if a[0][0] == "R"]
    annotation = ners + relations
    for a in annotation:
        # error annotations
        # TODO fix
        if len(a) < 2:
            continue
        annotation_entry: dict = dict()
        if a[0].startswith("T"):
            annotation_type = "NER"
        elif a[0].startswith("R"):
            annotation_type = "Relation"
        else:
            annotation_type = "Unknown"
        annotation_entry["type"] = annotation_type
        annotation_entry["code"] = annotation_code = a[0]

        annotation_body = a[1]
        annotation_body = annotation_body.replace("Arg1:", "").replace(
            "Arg2:", ""
        )
        annotation_body = annotation_body.split()
        annotation_entry["ann_type"] = annotation_body[0]
        if annotation_type == "NER":
            if len(a) <= 2:
                continue
            annotation_entry["start"] = int(annotation_body[1])
            annotation_entry["end"] = int(annotation_body[2])
            annotation_entry["text"] = a[2]
        else:
            for item_id, _ in enumerate(("subj", "obj")):
                item_id += 1
                for position in ("start", "end"):
                    annotation_entry[
                        f"{item_id}_{position}"
                    ] = annotation_entries[annotation_body[item_id]][position]
            annotation_entry["subj_ref"] = annotation_body[1]
            annotation_entry["obj_ref"] = annotation_body[2]
        annotation_entries[annotation_code] = annotation_entry
    annotation_values = list(annotation_entries.values())
    return annotation_values

def tokenize_and_align_labels(example, text_column_name="text", label_column_name="ner", padding=True, label_all_tokens=True, max_seq_length=512):
    # https://github.com/huggingface/transformers/blob/main/examples/pytorch/token-classification/run_ner.py#L421
    texts = example[text_column_name]
    if type(sents_df["text"]) == pd.core.series.Series:
        texts = list(texts.values)
    tokenized_inputs = tokenizer(
        texts,
        padding=padding,
        truncation=True,
        max_length=max_seq_length,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(example[label_column_name]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            new_label = [0] * len(id_to_label)
            if word_idx is None:
                label_ids.append(new_label)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                for ner in label[word_idx]:
                    new_label[label_to_id[ner]] = 1
                label_ids.append(new_label)
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                if label_all_tokens:
                    label_ids.append(b_to_i_label[label_to_id[label[word_idx]]])
                else:
                    label_ids.append(new_label)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [17]:
# def create_ner_dataset(glob_path="rured2/data/annotation_files/*.ann"):
    
glob_path="rured2/data/annotation_files/*.ann"
filenames = glob(glob_path)
text_data = dict()
for annotation_filename in filenames:
    text_filename = annotation_filename.replace(".ann", ".txt")
    annotation_values = get_annotation_from_file(annotation_filename)
    with open(text_filename) as f:
        text = f.read()
    sents = list(razdel.sentenize(text))
    prev_sent = None
    text_data[annotation_filename] = []
    attach_flag = False
    for sent in sents:
        if attach_flag and prev_sent:
            sent.text = prev_sent.text + " " + sent.text
            sent.start = prev_sent.start
            attach_flag = False
            prev_sent = None
        sent_tokens = list(razdel.tokenize(sent.text))
        sent_ners = [["O"] for t in sent_tokens]
        # sent_brat_codes = [["O"] for t in sent_tokens]
        sent_data: Dict = dict()
        sent_data["token"] = [t.text for t in sent_tokens]
        sent_data["text"] = sent.text
        # sent_data["relation"] = False
        # sent_data["relations"] = []
        for _, value in enumerate(annotation_values):
            annotation_code = value["code"]
            if value["type"] == "NER":
                start = value["start"]
                end = value["end"]
                ner_text = value["text"]
                ner_type = value["ann_type"]
                overlaps = check_pair_in_range(
                    start, end, range(sent.start, sent.stop)
                )
                if overlaps:
                    for s_t_i, s_t in enumerate(sent_tokens):
                        token_overlaps = check_pair_in_range(
                            start,
                            end,
                            range(
                                s_t.start + sent.start,
                                s_t.stop + sent.start,
                            ),
                        )
                        if token_overlaps:
                            if sent_ners[s_t_i] == ['O']:
                                sent_ners[s_t_i] = []
                            if ner_text.startswith(s_t.text):
                                sent_ners[s_t_i].append("B-" + ner_type)
                            else:
                                sent_ners[s_t_i].append("I-" + ner_type)

        sent_data["stanford_ner"] = sent_ners
        # sent_data["brat_codes"] = sent_brat_codes
        sent_data["start"] = sent.start
        sent_data["stop"] = sent.stop
        # all_data.append(sent_data)
        text_data[annotation_filename].append(sent_data)

In [18]:
sents = []
for filename, text in text_data.items():
    for sent in text:
        sent["filename"] = filename
        sents.append(sent)

In [19]:
label_to_id = {"O": 0}
id_to_label = {0: "O"}
new_label = 1

for sent in sents:
    for token_ners in sent["stanford_ner"]:
        for ner in token_ners:
            if ner not in label_to_id:
                label_to_id[ner] = new_label
                id_to_label[new_label] = ner
                new_label += 1
print(id_to_label)

{0: 'O', 1: 'B-PERSON', 2: 'B-NORP', 3: 'B-COMMODITY', 4: 'B-DATE', 5: 'I-DATE', 6: 'B-COUNTRY', 7: 'B-ECONOMIC_SECTOR', 8: 'I-ECONOMIC_SECTOR', 9: 'B-NEWS_SOURCE', 10: 'B-PROFESSION', 11: 'I-NEWS_SOURCE', 12: 'I-PERSON', 13: 'B-ORGANIZATION', 14: 'I-PROFESSION', 15: 'B-EVENT', 16: 'B-CITY', 17: 'B-GPE', 18: 'I-EVENT', 19: 'B-GROUP', 20: 'B-ORDINAL', 21: 'B-PRODUCT', 22: 'I-ORGANIZATION', 23: 'B-MONEY', 24: 'I-MONEY', 25: 'B-CURRENCY', 26: 'B-PERCENT', 27: 'I-PERCENT', 28: 'I-GROUP', 29: 'B-CARDINAL', 30: 'B-LAW', 31: 'I-LAW', 32: 'B-FAC', 33: 'I-FAC', 34: 'B-AGE', 35: 'I-CITY', 36: 'B-WORK_OF_ART', 37: 'I-WORK_OF_ART', 38: 'B-REGION', 39: 'I-REGION', 40: 'I-CARDINAL', 41: 'I-CURRENCY', 42: 'B-QUANTITY', 43: 'I-QUANTITY', 44: 'B-CRIME', 45: 'I-CRIME', 46: 'B-TRADE_AGREEMENT', 47: 'B-NATIONALITY', 48: 'B-FAMILY', 49: 'I-FAMILY', 50: 'I-PRODUCT', 51: 'B-TIME', 52: 'I-TIME', 53: 'I-COMMODITY', 54: 'B-APPLICATION', 55: 'I-APPLICATION', 56: 'I-COUNTRY', 57: 'B-AWARD', 58: 'I-AWARD', 59: 'I-

In [20]:
import json
with open("id_to_label.json", "w") as f:
    json.dump(id_to_label, f, ensure_ascii=False)

In [21]:
sents_df = pd.DataFrame(sents)
sents_df.head(5)

,token,text,stanford_ner,start,stop,filename
0,"[Гройсман, пообещал, дать, украинцам, собствен...",Гройсман пообещал дать украинцам собственный г...,"[[B-PERSON], [O], [O], [B-NORP], [O], [B-COMMO...",0,125,rured2/data/annotation_files/101144.ann
1,"[Об, этом, в, среду, ,, 28, декабря, ,, заявил...","Об этом в среду, 28 декабря, заявил премьер-ми...","[[O], [O], [B-DATE], [I-DATE], [I-DATE], [I-DA...",126,225,rured2/data/annotation_files/101144.ann
2,"[«, Мы, ставим, задачу, до, 2020, года, перейт...",«Мы ставим задачу до 2020 года перейти на макс...,"[[O], [O], [O], [O], [B-DATE], [I-DATE], [I-DA...",226,448,rured2/data/annotation_files/101144.ann
3,"[Он, призвал, потреблять, меньше, энергоресурс...","Он призвал потреблять меньше энергоресурсов, а...","[[O], [O], [O], [O], [B-COMMODITY], [O], [O], ...",449,582,rured2/data/annotation_files/101144.ann
4,"[23, декабря, министр, энергетики, и, угольной...",23 декабря министр энергетики и угольной промы...,"[[B-DATE], [I-DATE], [B-PROFESSION, B-NEWS_SOU...",583,717,rured2/data/annotation_files/101144.ann


In [22]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, use_fast=True)

/root/miniconda3/envs/python3.10/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [23]:
sents_train, sents_test = train_test_split(sents_df, test_size=0.1, random_state=42)

In [24]:
tokenized_train = tokenize_and_align_labels(sents_train, "token", "stanford_ner", label_all_tokens=False)
tokenized_test = tokenize_and_align_labels(sents_test, "token", "stanford_ner", label_all_tokens=False)

In [25]:
# hf_dataset = Dataset.from_dict(tokenized_inputs)
hf_train = Dataset.from_dict(tokenized_train)
hf_test = Dataset.from_dict(tokenized_test)

In [26]:
from torch import nn
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # predictions = (predictions > 0.5).astype(int)
    # f1_results = f1_score(labels,
    #                       (predictions > 0.5).astype(int),
    #                       average=None)
    # print("pred0", predictions[0])
    # print("pred0sampl", (predictions[0] > 0.5).astype(int))
    # print("label0", labels[0])

    # print("preds", predictions)
    # print("labels", labels)
    sigmoid = nn.Sigmoid()
    # print(labels.shape)
    # print(predictions.shape)
    f1_micro = 0
    f1_macro = 0
    output = dict()
    for i, l in enumerate(labels):
        sigmod_preds = (sigmoid(
            torch.Tensor(predictions[i])
            ).numpy() > 0.5
                        ).astype(int)
        f1_micro += f1_score(l,
                             sigmod_preds,
                             # (predictions > 0.5).astype(int),
                             average="micro",
                             zero_division=0)
    f1_micro /= predictions.shape[0]
    output['f1_micro'] = f1_micro
    macro_metrics = dict()
    for j in range(predictions.shape[2]):
        # sents; words; class
        col_labels = labels[:, :, j]
        col_preds = predictions[:, :, j]
        col_sigmoid_preds = (sigmoid(torch.Tensor(col_preds)).numpy() > 0.5).astype(int)
        for metric in (f1_score, recall_score, precision_score):
            metric_name = metric.__name__.replace("_score", "")
            for average in [
                    # "macro",
                    "micro"
            ]:
                score = metric(col_labels,
                               col_sigmoid_preds,
                               average=average,
                               zero_division=0
                              )
                output[f"{id_to_label[j]}_{metric_name}_{average}"] = score
                if metric_name not in macro_metrics:
                    macro_metrics[metric_name] = 0
                if j > 0:
                    macro_metrics[metric_name] += score
    for metric, score in macro_metrics.items():
        # do not include O
        output[f"{metric}_macro"] = score / (len(id_to_label) - 1)
    return output

class MultiLabelNERTrainer(Trainer):
    def __init__(self, *args, class_weights = None, **kwargs):
        super().__init__(*args, **kwargs)
        if class_weights is not None:
            class_weights = class_weights.to(self.args.device)
            logging.info(f"Using multi-label classification with class weights", class_weights)
        self.loss_fct = torch.nn.BCEWithLogitsLoss(weight=class_weights)

    def compute_loss(self, model, inputs, return_outputs=False):
        """
        How the loss is computed by Trainer. By default, all models return the loss in the first element.
        Subclass and override for custom behavior.
        """
        labels  = inputs.pop("labels")
        outputs = model(**inputs)
        # print(outputs)
        # this accesses predictions for tokens that aren't CLS, PAD, or the 2nd+ subword in a word
        # and simultaneously flattens the logits or labels
        # print("LOGITS", outputs.logits)
        # print("LABELS", labels)
        # print("labels 0", labels[0])
        # print("LOGITS 0", outputs.logits[0])
        # print("labels 0", labels[0].shape)
        # print("LOGITS 0", outputs.logits[0].shape)
        # flat_outputs = outputs.logits[labels > 0] 
        # flat_outputs = outputs.logits.view(8 * 340, -1)[(labels.sum(axis=2) > 2).view(-1)].view(8, -1, 76)
        flat_outputs = outputs.logits[labels.sum(axis=2) > 0]
        flat_labels  = labels[labels.sum(axis=2) > 0]
        # print("flat_outputs", flat_outputs.shape)
        # print("flat_labels", flat_labels.shape)
        try:
            loss = self.loss_fct(flat_outputs, flat_labels.float())
        except AttributeError:  # DataParallel
            loss = self.loss_fct(flat_outputs, flat_labels)

        return (loss, outputs) if return_outputs else loss

In [27]:
# Make sure your data is correctly formatted, e.g., 0 or 1 encoded for each label for each sample.

In [28]:
model = AutoModelForTokenClassification.from_pretrained(model_ckpt, num_labels=len(id_to_label))#.to('cuda')
model.config.label2id = label_to_id
model.config.id2label = id_to_label

Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/mdeberta-v3-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="multilabel_ner",
    save_strategy="epoch",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10000,
    weight_decay=0.01,
    save_total_limit=4,
    metric_for_best_model="f1_macro",
    load_best_model_at_end=True,
    fp16=True
)

trainer = MultiLabelNERTrainer(
    model=model,
    args=training_args,
    train_dataset=hf_train,
    eval_dataset=hf_test,
    tokenizer=tokenizer,
    # data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)],
)

# for param in model.parameters():
#     param.requires_grad = False

dataset_name = "rured2_model"
# # gradual unfreezing
# for i, param in enumerate(list(model.parameters())[::-1]):
#     param.requires_grad = True
#     if i % 40 != 0:
#         continue
#     print(i, len(param.shape), len(list(model.parameters())))
#     trainer.evaluate()
#     trainer.train()
#     results = trainer.evaluate()
#     prev_score = results["eval_f1_macro"]
#     best_model_checkpoint = None
#     if results["eval_f1_macro"] < prev_score and i >= 2:
#         print(f"loading previous best model: {best_model_checkpoint}")
#         trainer.state.best_model_checkpoint = best_model_checkpoint
#         trainer._load_best_model()
#     else:
#         prev_score = results["eval_f1_macro"]
#         best_model_checkpoint = f"{dataset_name}_step{i}"
#         print(f"saving new best model: {best_model_checkpoint}")
#         trainer.save_model(f"{dataset_name}_step{i}")

trainer.evaluate()
trainer.train()
trainer.train()
# trainer.save_model("multilabel_ner_rured2")

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


In [115]:
trainer.push_to_hub()

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.54k [00:00<?, ?B/s]

'https://huggingface.co/denis-gordeev/multilabel_ner/tree/main/'

In [111]:
trainer.save_model("bert_rured_training2")

In [41]:
!zip bert_rured_training.zip bert_rured_training/*
!cp bert_rured_training.zip drive/MyDrive/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


updating: bert_rured_training/added_tokens.json (stored 0%)
updating: bert_rured_training/config.json (deflated 69%)
updating: bert_rured_training/model.safetensors (deflated 27%)
updating: bert_rured_training/special_tokens_map.json (deflated 50%)
updating: bert_rured_training/spm.model (deflated 46%)
updating: bert_rured_training/tokenizer.json (deflated 76%)
updating: bert_rured_training/tokenizer_config.json (deflated 73%)
updating: bert_rured_training/training_args.bin (deflated 50%)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


cp: cannot create regular file 'drive/MyDrive/': No such file or directory


In [42]:
cp drive/MyDrive/bert_rured_training.zip ./

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


cp: cannot stat 'drive/MyDrive/bert_rured_training.zip': No such file or directory


In [ ]:
!unzip bert_rured_training.zip

In [ ]:
import json
with open("bert_rured_training/id_to_label.json") as f:
    id_to_label = json.load(f)
id_to_label = {int(k): v for k, v in id_to_label.items()}

In [116]:
model = AutoModelForTokenClassification.from_pretrained(
    f"denis-gordeev/rured2-ner-microsoft-mdeberta-v3-base") #.to('cuda')

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

/root/miniconda3/envs/python3.10/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [117]:
tokenizer = AutoTokenizer.from_pretrained(f"denis-gordeev/rured2-ner-microsoft-mdeberta-v3-base")

In [ ]:
model_desc = model_ckpt.replace("/", "_")
model = AutoModelForTokenClassification.from_pretrained(
    f"multilabel_ner_rured2_{model_ckpt}") #.to('cuda')
model.config.id2label = id_to_label

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [123]:
from torch import nn
def predict(text:str, glue_tokens=False, output_together=True, glue_words=True):
    sigmoid = nn.Sigmoid()
    tokenized = tokenizer(text)
    input_ids = torch.tensor(
            [tokenized["input_ids"]], dtype=torch.long
        ).to("cuda")
    token_type_ids = torch.tensor(
            [tokenized["token_type_ids"]], dtype=torch.long
        ).to("cuda")
    attention_mask = torch.tensor(
            [tokenized["attention_mask"]], dtype=torch.long
        ).to("cuda")
    preds = model(**{"input_ids": input_ids, "token_type_ids": token_type_ids, "attention_mask": attention_mask})
    logits = sigmoid(preds.logits)

    output_tokens = []
    output_preds = []
    for i, token in enumerate(input_ids[0]):
        if token > 3:
            class_ids = (logits[0][i] > 0.5).nonzero()
            if class_ids.shape[0] >= 1:
                class_names = [id_to_label[int(cl)] for cl in class_ids]
            else:
                class_names = [id_to_label[int(logits[0][i].argmax())]]
            converted_token = tokenizer.convert_ids_to_tokens([token])[0]
            new_word_bool = converted_token.startswith("▁")
            converted_token = converted_token.replace("▁", "")
            if glue_words and not(new_word_bool) and output_tokens:
                output_tokens[-1] += converted_token
            else:
                output_tokens.append(converted_token)
                output_preds.append(class_names)
        else:
            class_names = []
    if output_together:
        return [[output_tokens[t_i], output_preds[t_i]] for t_i in range(len(output_tokens))]
    return output_tokens, output_preds

In [35]:
trainer.evaluate()

{'eval_loss': 0.009618951939046383,
 'eval_f1_micro': 0.5836586590836932,
 'eval_O_f1_micro': 0.6370438459417226,
 'eval_O_recall_micro': 0.9241519095370173,
 'eval_O_precision_micro': 0.48604370634274974,
 'eval_B-PERSON_f1_micro': 0.963882618510158,
 'eval_B-PERSON_recall_micro': 0.9816091954022989,
 'eval_B-PERSON_precision_micro': 0.9467849223946785,
 'eval_B-NORP_f1_micro': 0.6190476190476191,
 'eval_B-NORP_recall_micro': 0.8666666666666667,
 'eval_B-NORP_precision_micro': 0.48148148148148145,
 'eval_B-COMMODITY_f1_micro': 0.755287009063444,
 'eval_B-COMMODITY_recall_micro': 0.946969696969697,
 'eval_B-COMMODITY_precision_micro': 0.628140703517588,
 'eval_B-DATE_f1_micro': 0.8386454183266933,
 'eval_B-DATE_recall_micro': 0.8470824949698189,
 'eval_B-DATE_precision_micro': 0.8303747534516766,
 'eval_I-DATE_f1_micro': 0.6419161676646707,
 'eval_I-DATE_recall_micro': 0.9492325855962219,
 'eval_I-DATE_precision_micro': 0.48492159227985526,
 'eval_B-COUNTRY_f1_micro': 0.615195071868583

In [124]:
texts = [
    "Газпром продает нефть Украине",
    "Согласно Reuters продажа газа Россией упала на 4%.",
    "IPhone 10 - последняя модель от Apple",
    "Рогозин объявил о завершении военной операции в Сирии",
    "Число рабочих мест в Санкт-Петербурге будет увеличено на 1000 в связи с вовзедением завода Уралзаводвагмаш"
]
for text in texts:
    print(predict(text))
    print(("-" * 80 + "\n")*2)

[['Газпром', ['B-ORGANIZATION']], ['продает', ['O']], ['нефть', ['B-COMMODITY']], ['Украине', ['B-COUNTRY']]]
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------

[['Согласно', ['O']], ['Reuters', ['B-ORGANIZATION']], ['продажа', ['O']], ['газа', ['B-COMMODITY']], ['Россией', ['B-COUNTRY']], ['упала', ['O']], ['на', ['B-PERCENT']], ['4%.', ['I-PERCENT']]]
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------

[['IPhone', ['B-PRODUCT']], ['10', ['I-PRODUCT']], ['-', ['O']], ['последняя', ['O']], ['модель', ['O']], ['от', ['O']], ['Apple', ['B-ORGANIZATION']]]
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------

[['Рогозин', ['B-PERSON', 'B-NEWS_SOURCE']], ['объявил

In [96]:
predict(
    'Компания «Читинские ключи» получила господдержку и запустила производство «Колы» для сохранения рабочих мест, '
    'сообщили в пресс-службе министерства экономического развития Забайкальского края. По словам замминистра экономического '
    'развития Забайкалья Дениса Рысева, у &laquo;Колы&raquo; особенный вкус. На производстве уверяют, что используется секретный '
    'ингредиент. Как и у той &laquo;Кока-колы&raquo;, известной на весь мир. Благодаря запуску нового производства компании удалось '
    'сохранить 241 сотрудника, сообщает РИА &laquo;Новости&raquo;. Уточняется, что первая партия &laquo;Колы&raquo; – 900 упаковок '
    'была изготовлена 5 октября. В этот же день 600 упаковок купили предприниматели региона для продажи в магазинах. &laquo;Это '
    'ожидаемый продукт в условиях импортозамещения. Скучали по &laquo;Кока-коле&raquo;? Вот же она –&nbsp;только еще лучше, своя! '
    'Гарантийный фонд Забайкалья стал поручителем по кредиту для компании &laquo;Читинские ключи&raquo;. Сумма поручительства 25 '
    'миллионов рублей, сумма кредита –&nbsp;почти 50 миллионов рублей&raquo;, – добавил Рысев. Ранее сообщалось, что в Московской '
    'области планируется запустить новое производство компрессоров и фильтров для жидкостей, этим займется компания &laquo;Полинет&raquo;.',)

[['Компания', ['B-ORGANIZATION']],
 ['«Читинские', ['I-ORGANIZATION']],
 ['ключи»', ['I-ORGANIZATION']],
 ['получила', ['O']],
 ['господдержку', ['I-INVESTMENT_PROGRAM']],
 ['и', ['O']],
 ['запустила', ['O']],
 ['производство', ['B-ECONOMIC_SECTOR']],
 ['«Колы»', ['B-PRODUCT']],
 ['для', ['O']],
 ['сохранения', ['O']],
 ['рабочих', ['O']],
 ['мест,', ['O']],
 ['сообщили', ['B-NEWS_SOURCE']],
 ['в', ['I-NEWS_SOURCE', 'B-ORGANIZATION']],
 ['пресс-службе', ['I-NEWS_SOURCE', 'I-ORGANIZATION']],
 ['министерства', ['I-NEWS_SOURCE', 'B-ORGANIZATION']],
 ['экономического', ['I-NEWS_SOURCE', 'I-ORGANIZATION']],
 ['развития', ['I-NEWS_SOURCE', 'I-ORGANIZATION']],
 ['Забайкальского', ['I-NEWS_SOURCE', 'B-REGION']],
 ['края.', ['I-NEWS_SOURCE', 'I-REGION']],
 ['По', ['B-NEWS_SOURCE']],
 ['словам', ['I-NEWS_SOURCE']],
 ['замминистра', ['B-PROFESSION', 'I-NEWS_SOURCE']],
 ['экономического', ['I-NEWS_SOURCE', 'I-PROFESSION']],
 ['развития', ['I-NEWS_SOURCE', 'I-PROFESSION']],
 ['Забайкалья', ['I-NEWS

In [92]:
predict('В Новосибирске задержали безработного жителя Томска, который продавал наркотики через интернет-магазин')

[['В', ['O']],
 ['Новосибирске', ['B-CITY']],
 ['задержали', ['O']],
 ['безработного', ['O']],
 ['жителя', ['O']],
 ['Томска,', ['B-CITY']],
 ['который', ['O']],
 ['продавал', ['I-CRIME']],
 ['наркотики', ['I-CRIME']],
 ['через', ['I-CRIME']],
 ['интернет-магазин', ['B-FAC']]]

In [125]:
predict('В России стали заменять одежду брендов H&M и Zara дорогими аналогами из Турции. Об этом сообщает издание NEWS.ru. По словам анонимного источника портала, среди подобных альтернатив оказались такие турецкие марки, как Perspective, Ketroy и Mexx, магазины которого закрылись в стране в 2017 году из-за нерентабельности.')

[['В', ['O']],
 ['России', ['B-COUNTRY']],
 ['стали', ['O']],
 ['заменять', ['O']],
 ['одежду', ['B-PRODUCT']],
 ['брендов', ['O']],
 ['H&M', ['B-ORGANIZATION']],
 ['и', ['O']],
 ['Zara', ['B-ORGANIZATION']],
 ['дорогими', ['O']],
 ['аналогами', ['O']],
 ['из', ['O']],
 ['Турции.', ['B-COUNTRY']],
 ['Об', ['O']],
 ['этом', ['O']],
 ['сообщает', ['B-NEWS_SOURCE']],
 ['издание', ['I-NEWS_SOURCE']],
 ['NEWS.ru.', ['I-NEWS_SOURCE', 'B-ORGANIZATION']],
 ['По', ['B-NEWS_SOURCE']],
 ['словам', ['I-NEWS_SOURCE']],
 ['анонимного', ['I-NEWS_SOURCE']],
 ['источника', ['I-NEWS_SOURCE']],
 ['портала,', ['I-NEWS_SOURCE']],
 ['среди', ['O']],
 ['подобных', ['O']],
 ['альтернатив', ['O']],
 ['оказались', ['O']],
 ['такие', ['O']],
 ['турецкие', ['B-COUNTRY']],
 ['марки,', ['O']],
 ['как', ['O']],
 ['Perspective,', ['B-PRODUCT']],
 ['Ketroy', ['B-PRODUCT']],
 ['и', ['O']],
 ['Mexx,', ['B-PRODUCT']],
 ['магазины', ['B-FAC']],
 ['которого', ['O']],
 ['закрылись', ['O']],
 ['в', ['O']],
 ['стране', ['O']]

In [45]:
import huggingface_hub

In [48]:
huggingface_hub.login()

In [49]:
from huggingface_hub import create_repo
create_repo("denis-gordeev/rured2-ner-microsoft-mdeberta-v3-base")

RepoUrl('https://huggingface.co/denis-gordeev/rured2-ner-microsoft-mdeberta-v3-base', endpoint='https://huggingface.co', repo_type='model', repo_id='denis-gordeev/rured2-ner-microsoft-mdeberta-v3-base')

In [107]:
from glob import glob
model_files = glob("bert_rured_training/*")

In [108]:
from huggingface_hub import HfApi
api = HfApi()
for filename in model_files:
    api.upload_file(
        path_or_fileobj=filename,
        path_in_repo=filename.split("/")[-1],
        repo_id="denis-gordeev/rured2-ner-microsoft-mdeberta-v3-base",
    )

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.54k [00:00<?, ?B/s]

In [98]:
ls bert_rured_training

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


added_tokens.json  special_tokens_map.json  tokenizer_config.json
config.json        spm.model                training_args.bin
model.safetensors  tokenizer.json


In [ ]:
!cp id_to_label.json multilabel_ner_rured2/
!zip multilabel_ner_rured2_sept30_2022.zip multilabel_ner_rured2/*
!cp multilabel_ner_rured2_sept30_2022.zip drive/MyDrive 